# Fit Lines to Segmentations
Here we fit lines to the predicted segmentations. To do so we first must run some analysis code to find the bounding box of each corresponding to true positive, false positive etc. This is in a separate repo but could also be done here. We then look at each segmentated object one at a time and fit a line to its voxels. We then project all these voxels along this line to find the extents of the line to make it a line segment. We then write this out as neuroglancer annotations. You can also write these out however you would like, you would just have to update the downstream code accordingly.

In [2]:
import numpy as np
from neuroglancer.write_annotations import AnnotationWriter
from neuroglancer import AnnotationPropertySpec
from neuroglancer.coordinate_space import CoordinateSpace
import numpy as np
import numpy as np
from funlib.geometry import Roi
from funlib.persistence import open_ds
import pandas as pd
from tqdm import tqdm
from importlib import reload
import utils.utils
from funlib.geometry import Coordinate

reload(utils.utils)
from utils.utils import open_ds_tensorstore, to_ndarray_tensorstore


def find_min_max_projected_points(points, line_point, line_direction):
    # chatgpt
    line_direction = line_direction / np.linalg.norm(
        line_direction
    )  # Normalize direction vector

    # Calculate the vector from line_point to each point
    point_vectors = points - line_point

    # Calculate the projection scalar for each point using dot product and broadcasting
    projection_scalars = np.sum(point_vectors * line_direction, axis=1)

    # Calculate the projected points for each point
    projected_points = line_point + projection_scalars[:, np.newaxis] * line_direction

    # Find the minimum and maximum projection scalar indices
    min_projection_idx = np.argmin(projection_scalars)
    max_projection_idx = np.argmax(projection_scalars)

    return projected_points[min_projection_idx], projected_points[max_projection_idx]


def fit_line_to_points(points, line_origin=0):
    # fit line to object voxels
    _, _, vv = np.linalg.svd(points - np.mean(points, axis=0), full_matrices=False)
    line_direction = vv[0]

    # find endpoints of line segment so that we can write it as neuroglancer annotations
    start_point, end_point = find_min_max_projected_points(
        points * 8 + 4 + roi.begin, line_origin, line_direction
    )

    return start_point, end_point


plasmodesmata_df = pd.read_csv(
    "/nrs/cellmap/ackermand/cellmap/analysisResults/leaf-gall/jrc_22ak351-leaf-3m.n5/fragments_relabeled.csv"
)
ds = open_ds(
    "/nrs/cellmap/ackermand/cellmap/leaf-gall/jrc_22ak351-leaf-3m.n5",
    "fragments_relabeled",
)
ts = open_ds_tensorstore(
    "/nrs/cellmap/ackermand/cellmap/leaf-gall/jrc_22ak351-leaf-3m.n5/fragments_relabeled"
)

# since it is arbitrary to have endpoints for line segment in terms of fitting, will just fit a line and then truncate it
annotation_writer = AnnotationWriter(
    CoordinateSpace(names=("x", "y", "z"), scales=(1, 1, 1), units="nm"),
    annotation_type="line",
    properties=[
        AnnotationPropertySpec(id="identifier", type="uint16"),
    ],
)

# uu, dd, vv = np.linalg.svd(data - datamean, full_matrices=False)
for _, row in tqdm(plasmodesmata_df.iterrows(), total=len(plasmodesmata_df)):
    # object id, bounding box and center of mass information, calculated beforehand
    id = row["Object ID"]
    cube_min = np.array([row[f"MIN {d} (nm)"] for d in ["X", "Y", "Z"]])
    cube_max = np.array([row[f"MAX {d} (nm)"] for d in ["X", "Y", "Z"]])
    com = np.array([row[f"COM {d} (nm)"] for d in ["X", "Y", "Z"]])

    # define an roi to actually ecompass the bounding box
    roi = Roi(cube_min - 8, (cube_max - cube_min) + 16)

    # only look at pixels corresponding to current object
    cube = to_ndarray_tensorstore(
        ts, roi, ds.voxel_size, Coordinate(ds.roi.begin[::-1])
    )
    data = np.column_stack(np.where(cube == id))
    start_point, end_point = fit_line_to_points(data, com)
    # # fit line to object voxels
    # uu, dd, vv = np.linalg.svd(data - np.mean(data, axis=0), full_matrices=False)
    # line_direction = vv[0]
    # line_origin = com

    # # find endpoints of line segment so that we can write it as neuroglancer annotations
    # start_point, end_point = find_min_max_projected_points(
    #     data * 8 + 4 + roi.begin, line_origin, line_direction
    # )

    # write out lines as neuroglancer annotations
    # annotation_writer.add_line(
    #     point_a=start_point[::-1],
    #     point_b=end_point[::-1],
    #     id=int(id),
    #     identifier=int(id),
    # )

    # annotation_writer.write(
    #     f"/groups/cellmap/cellmap/ackermand/neuroglancer_annotations/leaf-gall/forAnnotators/{yaml_name}/{ds_name}"
    # )

  0%|          | 258/391068 [00:12<5:21:13, 20.28it/s]


KeyboardInterrupt: 

In [55]:
import utils.utils
from funlib.geometry import Coordinate

reload(utils.utils)
from utils.utils import open_ds_tensorstore, to_ndarray_tensorstore

cube = to_ndarray_tensorstore(ts, roi, ds.voxel_size, Coordinate(ds.roi.begin[::-1]))

roi_slices slice(2, 42, None), valid_slices slice(2, 42, None), padded_slicesslice(-2, 40, None), (0, 0, 0)


ValueError: could not broadcast input array from shape (23,40,22) into shape (23,2,22)

In [54]:
roi, ds.roi

([174888:175072, 12816:13136, 54616:54792] (184, 320, 176),
 [22400:73728, 12800:38528, 0:261120] (51328, 25728, 261120))

In [2]:
import numpy as np
from neuroglancer.write_annotations import AnnotationWriter
from neuroglancer import AnnotationPropertySpec
from neuroglancer.coordinate_space import CoordinateSpace
import numpy as np
import numpy as np
from funlib.geometry import Roi
from funlib.persistence import open_ds
import pandas as pd
from tqdm import tqdm
from utils.utils import open_ds_tensorstore, to_ndarray_tensorstore


def find_min_max_projected_points(points, line_point, line_direction):
    # chatgpt
    line_direction = line_direction / np.linalg.norm(
        line_direction
    )  # Normalize direction vector

    # Calculate the vector from line_point to each point
    point_vectors = points - line_point

    # Calculate the projection scalar for each point using dot product and broadcasting
    projection_scalars = np.sum(point_vectors * line_direction, axis=1)

    # Calculate the projected points for each point
    projected_points = line_point + projection_scalars[:, np.newaxis] * line_direction

    # Find the minimum and maximum projection scalar indices
    min_projection_idx = np.argmin(projection_scalars)
    max_projection_idx = np.argmax(projection_scalars)

    return projected_points[min_projection_idx], projected_points[max_projection_idx]


plasmodesmata_df = pd.read_csv(
    "/nrs/cellmap/ackermand/cellmap/analysisResults/leaf-gall/jrc_22ak351-leaf-3m.n5/fragments_relabeled.csv"
)
ds = open_ds(
    "/nrs/cellmap/ackermand/cellmap/leaf-gall/jrc_22ak351-leaf-3m.n5",
    "fragments_relabeled",
)

In [10]:
ds.roi.begin

(22400, 12800, 0)